In [26]:
from huggingface_hub import notebook_login,login
notebook_login()

In [27]:
! pip install transformers peft datasets

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import PeftModel, PeftConfig,peft_model
import torch
import os
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm

device='cuda'
model_name_or_path='bigscience/bloomz-7b1'
tokenizer_name_or_path='bigscience/bloomz-7b1'
dataset_name='twitter_complaints'
text_column='Tweet text'
label_column='text_label'
max_length=64
lr=1e-3
num_epochs=50
batch_size=8

In [29]:
dataset = load_dataset("ought/raft", dataset_name)
# Get label names and replace underscore with spaces
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]

# Map label IDs to label names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
print(dataset)
dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 3399
    })
})


{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [30]:
# Data Preprocessing
tokenizer=AutoTokenizer.from_pretrained(model_name_or_path)

# Ensuring pad_token_id is defined
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id=tokenizer.eos_token_id
target_max_length=max([len(tokenizer(class_label)['input_ids'])for class_label in classes])
print(target_max_length)

def preprocess_function(examples):
    batch_size=len(examples[text_column])
    inputs=[f"{text_column}:{x} Label :" for x in examples[text_column]]
    targets=[str(x) for x in examples[label_column]]
    model_inputs=tokenizer(inputs)
    labels=tokenizer(targets, add_special_tokens=False)
    for i in range(batch_size):
        sample_input_ids=model_inputs['input_ids'][i]
        label_input_ids=labels['input_ids'][i] + [tokenizer.eos_token_id]
        # Append Targets to inputs
        model_inputs['input_ids'][i]= sample_input_ids + label_input_ids
        # Prepare the training labels
        labels['input_ids'][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs['attention_mask'][i]=[1] * len(model_inputs['input_ids'][i])


    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

3


Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [ ]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    return model_inputs


processed_datasets = dataset.map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

eval_dataset = processed_datasets["train"]
test_dataset = processed_datasets["test"]

eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
print(next(iter(eval_dataloader)))
print(next(iter(test_dataloader)))


In [32]:
from peft import PeftModel, PeftConfig,peft_model
device_map={'':'cpu'}
max_memory = { "cpu": "30GB"}
peft_model_id = "OmBhandwalkar/twitter_complaints_bigscience_bloomz-7b1_LORA_CAUSAL_LM"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, max_memory=max_memory)


In [48]:
model = PeftModel.from_pretrained(model, peft_model_id, max_memory=max_memory)

adapter_model.safetensors:   0%|          | 0.00/15.7M [00:00<?, ?B/s]

In [50]:
model.eval()
i=89

inputs=tokenizer(f"{text_column} : {dataset['test'][i]['Tweet text']} Label : ",return_tensors='pt')
print(dataset['test'][i]['Tweet text'])
print(inputs)

with torch.no_grad():
    outputs=model.generate(input_ids=inputs['input_ids'], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(),skip_special_tokens=True))


@HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again.
{'input_ids': tensor([[227985,   5484,    915,   2566, 216744,     38,   1316,     54,  42705,
          32465,  52166,   9440,   1809,   3784,  88483,   9411,    368,  84342,
           4451,     17,    473,   2152,  11705,  82406,    267,  51591,   5734,
             17,  77658,    915,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}
tensor([[227985,   5484,    915,   2566, 216744,     38,   1316,     54,  42705,
          32465,  52166,   9440,   1809,   3784,  88483,   9411,    368,  84342,
           4451,     17,    473,   2152,  11705,  82406,    267,  51591,   5734,
             17,  77658,    915,    210,  16449,   5952,      2]])
['Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label 

In [51]:
model.eval()
eval_preds = []
for _, batch in enumerate(tqdm(eval_dataloader)):
    batch = {k: v for k, v in batch.items() if k != "labels"}
    with torch.no_grad():
        outputs = model.generate(**batch, max_new_tokens=10)
    preds = outputs[:, max_length:].detach().cpu().numpy()
    eval_preds.extend(tokenizer.batch_decode(preds, skip_special_tokens=True))

100%|██████████| 7/7 [13:30<00:00, 115.83s/it]


In [52]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["train"][label_column]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=}")
print(f"{eval_preds[:10]=}")
print(f"{dataset['train'][label_column][:10]=}")


accuracy=98.0
eval_preds[:10]=['no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint', 'no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint']
dataset['train'][label_column][:10]=['no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint', 'no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint']


In [53]:
model.eval()
test_preds = []

for _, batch in enumerate(tqdm(test_dataloader)):
    batch = {k: v for k, v in batch.items() if k != "labels"}
    with torch.no_grad():
        outputs = model.generate(**batch, max_new_tokens=10)
    preds = outputs[:, max_length:].detach().cpu().numpy()
    test_preds.extend(tokenizer.batch_decode(preds, skip_special_tokens=True))
    if len(test_preds) > 100:
        break
test_preds

  3%|▎         | 12/425 [28:06<16:07:13, 140.52s/it]


['no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'complaint',
 'complaint',
 'complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'complaint',
 'complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'n